In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import math
import xlsxwriter
from urllib.error import HTTPError

In [2]:
stocks = pd.read_csv('/workspaces/codespaces-jupyter/data/sp_500_stocks.csv')
stocks

Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
500    YUM
501    ZBH
502   ZBRA
503   ZION
504    ZTS

[505 rows x 1 columns]

In [3]:
msft = yf.Ticker('MSFT')
msft_one_month = msft.history(period='1mo')
msft_info = msft.info
msft_market_cap = msft_info['marketCap']
msft_close = msft_one_month['Close']

In [4]:
def get_price_info(ticker:str) -> pd.DataFrame:
    try:
        tick = yf.Ticker(ticker)
        tick_info = tick.info
    except Exception as e:
        return [None, None]
    
    try:
        market_cap = tick_info['marketCap']
    except KeyError as e:
        market_cap = None

    try:
        current_price = tick_info['currentPrice']
    except KeyError as e:
        current_price = None
    
    return [market_cap, current_price]

In [25]:
my_cols = ['ticker', 'price', 'market_cap', 'no_of_shares']
final_df = pd.DataFrame(columns=my_cols)

# i = 0
for ind, ticker in stocks.iterrows():
    # i+=1
    symbol = ticker.values[0]
    # price_info = get_price_info(symbol)
    # market_cap = price_info[0]
    # current_price = price_info[1]
    market_cap, current_price = get_price_info(symbol)
    to_append = pd.Series(
        [
            symbol,
            current_price,
            market_cap,
            'N/A'
        ],
        index=my_cols
    )
    final_df = pd.concat([final_df,to_append.to_frame().T], ignore_index=True)
    # if i == 5:
    #     break
final_df

ticker     price     market_cap no_of_shares
0        A   122.025    36043255808          N/A
1      AAL    16.655    10881743872          N/A
2      AAP     74.38     4421436928          N/A
3     AAPL  195.9759  3082446438400          N/A
4     ABBV   148.975   262835126272          N/A
..     ...       ...            ...          ...
500    YUM    137.69    38565179392          N/A
501    ZBH   138.045    28822138880          N/A
502   ZBRA  306.4125    15758886912          N/A
503   ZION     37.94     5620582912          N/A
504    ZTS    187.26    86535094272          N/A

[505 rows x 4 columns]

In [36]:
portfolio_size = 1000

val = float(portfolio_size)
position_size = val / len(final_df)
print(position_size)

final_df['no_of_shares'] = position_size / final_df['price']
print(final_df)
final_df['no_of_shares'].fillna(0, inplace=True)
print(final_df)
# final_df['no_of_shares'] = final_df['no_of_shares'].apply(lambda x:math.floor(x))


1.9801980198019802
    ticker     price     market_cap no_of_shares
0        A   122.025    36043255808     0.016228
1      AAL    16.655    10881743872     0.118895
2      AAP     74.38     4421436928     0.026623
3     AAPL  195.9759  3082446438400     0.010104
4     ABBV   148.975   262835126272     0.013292
..     ...       ...            ...          ...
500    YUM    137.69    38565179392     0.014382
501    ZBH   138.045    28822138880     0.014345
502   ZBRA  306.4125    15758886912     0.006463
503   ZION     37.94     5620582912     0.052193
504    ZTS    187.26    86535094272     0.010575

[505 rows x 4 columns]
    ticker     price     market_cap  no_of_shares
0        A   122.025    36043255808      0.016228
1      AAL    16.655    10881743872      0.118895
2      AAP     74.38     4421436928      0.026623
3     AAPL  195.9759  3082446438400      0.010104
4     ABBV   148.975   262835126272      0.013292
..     ...       ...            ...           ...
500    YUM    137.6

In [31]:
final_df

ticker     price     market_cap  no_of_shares
0        A   122.025    36043255808             0
1      AAL    16.655    10881743872             0
2      AAP     74.38     4421436928             0
3     AAPL  195.9759  3082446438400             0
4     ABBV   148.975   262835126272             0
..     ...       ...            ...           ...
500    YUM    137.69    38565179392             0
501    ZBH   138.045    28822138880             0
502   ZBRA  306.4125    15758886912             0
503   ZION     37.94     5620582912             0
504    ZTS    187.26    86535094272             0

[505 rows x 4 columns]